In [47]:
import re

import pandas as pd

In [48]:
ev_path = r"electric-car-stocks\electric-car-stocks.csv"
countries_path = r"paises_region.csv"
cars_path = r"Total-World-vehicles-in-use-2020.xlsx"

In [49]:
df_ev = pd.read_csv(ev_path)
df_countries = pd.read_csv(countries_path)
df_cars = pd.read_excel(cars_path, skiprows=3)

In [50]:
df_ev = df_ev[df_ev['Year'] == 2020]

In [51]:
df_cars.dropna(subset=['2015'], inplace=True)
df_cars.drop(columns=['Unnamed: 0', '2015', 'Average Annual Growth Rate\n 2015-2020'], inplace=True)
df_cars["REGIONS/COUNTRIES"] = df_cars["REGIONS/COUNTRIES"].str.title()
df_cars[2020] = df_cars[2020].astype(int)
df_cars['Number of vehicles per 1000 inhabitants'] = df_cars['Number of vehicles per 1000 inhabitants'].astype(int)

df_cars[2020] = df_cars[2020] * 1000
df_cars['Number of vehicles per 1000 inhabitants'] = df_cars['Number of vehicles per 1000 inhabitants'] * 1000

countries_map = {
    "Czech Republic": "Czechia",
    "Egypt": "Egypt, Arab Rep.",
    "Iran": "Iran, Islamic Rep.",
    "Russia": "Russian Federation",
    "Slovakia": "Slovak Republic",
    "South Korea": "Korea, Rep.",
    "Syria": "Syrian Arab Republic",
    "Turkey": "Turkiye",
    "United States Of America": "United States",
    "Venezuela": "Venezuela, RB",
    "Vietnam": "Viet Nam"
}

df_cars["REGIONS/COUNTRIES"] = df_cars["REGIONS/COUNTRIES"].replace(countries_map)
df_ev["Entity"] = df_ev["Entity"].replace(countries_map)

In [52]:
regiones = [
    "Africa Eastern and Southern",
    "Africa Western and Central",
    "Arab World",
    "Caribbean small states",
    "Central Europe and the Baltics",
    "Channel Islands",
    "Early-demographic dividend",
    "East Asia & Pacific",
    "Euro area",
    "Europe & Central Asia",
    "European Union",
    "Fragile and conflict affected situations",
    "Heavily indebted poor countries",
    "High income",
    "IBRD only",
    "IDA & IBRD total",
    "IDA blend",
    "IDA only",
    "IDA total",
    "Late-demographic dividend",
    "Latin America & Caribbean",
    "Latin America & the Caribbean",
    "Least developed countries: UN classification",
    "Low & middle income",
    "Low income",
    "Lower middle income",
    "Middle East, North Africa, Afghanistan & Pakistan",
    "Middle income",
    "North America",
    "Not classified",
    "OECD members",
    "Other small states",
    "Pacific island small states",
    "Post-demographic dividend",
    "Pre-demographic dividend",
    "Small states",
    "South Asia",
    "Sub-Saharan Africa",
    "Upper middle income",
    "World"
]

In [53]:
df_countries['country_1'] = df_countries['country']
df_countries['country_1']  = df_countries['country_1'].str.replace(r'\s*\(.*?\)', '', regex=True)
df_countries = df_countries[~df_countries.country_1.isin(regiones)]

In [54]:
merge1 = df_countries.merge(df_cars, left_on='country_1', right_on='REGIONS/COUNTRIES', how='outer')
merge1 = merge1.dropna(subset=["country_1"])

In [55]:
merge1 = merge1.copy()

# Rellenar columna '2020' con el promedio por región
regional_mean_2020 = (
    merge1.groupby("region")[2020]
    .transform(lambda x: x.mean(skipna=True))
)
merge1[2020] = merge1[2020].fillna(regional_mean_2020).astype(int)


# Rellenar columna 'Number of vehicles per 1000 inhabitants' con el promedio por región
regional_mean_vehicles = (
    merge1.groupby("region")["Number of vehicles per 1000 inhabitants"]
    .transform(lambda x: x.mean(skipna=True))
)
merge1["Number of vehicles per 1000 inhabitants"] = merge1["Number of vehicles per 1000 inhabitants"].fillna(regional_mean_vehicles).astype(int)

In [56]:
merge2 = merge1.merge(df_ev, left_on='country_1', right_on='Entity', how='outer')
merge2

,iso3c,country,region,country_1,REGIONS/COUNTRIES,2020,Number of vehicles per 1000 inhabitants,Entity,Code,Year,Electric car stocks
0,AFG,Afghanistan,MEA,Afghanistan,NaN,6229600.0,233800.0,NaN,NaN,NaN,NaN
1,ALB,Albania,ECS,Albania,NaN,14664172.0,530379.0,NaN,NaN,NaN,NaN
2,DZA,Algeria,MEA,Algeria,Algeria,6239000.0,143000.0,NaN,NaN,NaN,NaN
3,ASM,American Samoa,EAS,American Samoa,NaN,50952500.0,387200.0,NaN,NaN,NaN,NaN
4,AND,Andorra,ECS,Andorra,NaN,14664172.0,530379.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
215,PSE,West Bank and Gaza,MEA,West Bank and Gaza,NaN,6229600.0,233800.0,NaN,NaN,NaN,NaN
216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,World,OWID_WRL,2020.0,10200000.0
217,YEM,"Yemen, Rep.",MEA,"Yemen, Rep.",NaN,6229600.0,233800.0,NaN,NaN,NaN,NaN
218,ZMB,Zambia,SSF,Zambia,NaN,10971500.0,115000.0,NaN,NaN,NaN,NaN


In [57]:
valor =  int(merge2.loc[merge2['Entity'] == 'Rest of World', 'Electric car stocks'].values[0] / merge2[merge2['Electric car stocks'].isna()].shape[0])
valor

575

In [58]:
merge2["Electric car stocks"] = merge2["Electric car stocks"].fillna(valor)
merge2 = merge2.rename(columns={2020: "Total motor vehicles", "Number of vehicles per 1000 inhabitants": "Vehicles per 1000 inhabitants", "Electric car stocks": "Electric cars in use"})

In [60]:
merge2 = merge2.copy()
merge2['EV_adoption'] = (merge2['Electric cars in use'] / merge2['Total motor vehicles'] * 100).round(2)
merge2 = merge2[['country', 'region', 'Total motor vehicles', 'Electric cars in use', 'EV_adoption']]
merge2 = merge2.dropna(subset=["country"])

In [62]:
merge2.sort_values(by='EV_adoption', ascending=False)

,country,region,Total motor vehicles,Electric cars in use,EV_adoption
148,Norway,ECS,3416000.0,490000.0,14.34
189,Sweden,ECS,5637000.0,176000.0,3.12
140,Netherlands,ECS,10248000.0,290000.0,2.83
52,Denmark,ECS,3147000.0,62000.0,1.97
68,Finland,ECS,3191000.0,55700.0,1.75
...,...,...,...,...,...
203,Tuvalu,EAS,50952500.0,575.0,0.00
210,Uzbekistan,ECS,14664172.0,575.0,0.00
209,Uruguay,LCN,15637250.0,575.0,0.00
214,Virgin Islands (U.S.),LCN,15637250.0,575.0,0.00


In [63]:
merge2.to_csv("ev_adoption_v2.csv", index=False, encoding="latin")